In [118]:
from data import load, get_training_data

load(sqlContext, spark)

In [ ]:
splits = df.randomSplit([0.8, 0.2])
train = splits[0].cache()
test = splits[1].cache()

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler


artistInd = StringIndexer(inputCol="artist", outputCol="artistIndex").setHandleInvalid("skip")
titleInd = StringIndexer(inputCol="title", outputCol="titleIndex").setHandleInvalid("skip")
dayOfWeekInd = StringIndexer(inputCol="day_of_week", outputCol="dayOfWeekIndex").setHandleInvalid("skip")
assembler = VectorAssembler(inputCols=["artistIndex", "dayOfWeekIndex", "titleIndex", "hour", "day", "month", "year","currentrank"], outputCol="features")

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

lr = LogisticRegression(maxIter=10, threshold=0.1)
pipeline = Pipeline().setStages([artistInd, titleInd, dayOfWeekInd, assembler, lr])

model = pipeline.fit(train)

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

results = model.transform(test)
predictionsAndLabels = results.select("prediction", "label") \
                              .rdd.map(lambda r: (float(r["prediction"]), float(r["label"])))
metrics = BinaryClassificationMetrics(predictionsAndLabels)
print "AUC: {0}".format(metrics.areaUnderROC)
print "AUP: {0}".format(metrics.areaUnderPR)


In [ ]:
from pyspark.sql.functions import udf, col
results.filter(col("prediction") == 1.0).take(20)

In [4]:
from data import load, get_training_data
from datetime import datetime
load(sqlContext, spark)

In [5]:
df = get_training_data(spark, datetime(2017,3,1))
df = df.unionAll(get_training_data(spark, datetime(2017,2,22))) \
       .unionAll(get_training_data(spark, datetime(2017,2,15))) \
      .unionAll(get_training_data(spark, datetime(2017,2,8)))

+--------------------+--------------------+----+-----+----+-----------------+--------+-----------+-----+
|              artist|               title|year|month|week|weekly_play_count|nextrank|currentrank|label|
+--------------------+--------------------+----+-----+----+-----------------+--------+-----------+-----+
|             temples|strange or be for...|2017|    3|   9|               11|       0|          0|    0|
|         jens lekman|      evening prayer|2017|    3|   9|               11|       0|          0|    0|
|         marian hill|                down|2017|    3|   9|               10|      16|         18|    1|
|         mac demarco|          my old man|2017|    3|   9|               10|       0|          0|    0|
|           tim darcy| tall glass of water|2017|    3|   9|               10|       0|          0|    0|
|       the lumineers|              angela|2017|    3|   9|               10|       0|          0|    0|
|    guided by voices|         hiking skin|2017|    3| 

In [6]:
from data import get_training_data   
df = get_training_data(spark, datetime(2017,3,1))
df = df.unionAll(get_training_data(spark, datetime(2017,2,22))) \
  .unionAll(get_training_data(spark, datetime(2017,2,15))) \
  .unionAll(get_training_data(spark, datetime(2017,2,8))) \
  .unionAll(get_training_data(spark, datetime(2017,2,1)))
df.dropna()
splits = df.randomSplit([0.8, 0.2])
train = splits[0].cache()
test = splits[1].cache()

from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler


artistInd = StringIndexer(inputCol="artist", outputCol="artistIndex").setHandleInvalid("skip")
titleInd = StringIndexer(inputCol="title", outputCol="titleIndex").setHandleInvalid("skip")
assembler = VectorAssembler(inputCols=["artistIndex","titleIndex", "weekly_play_count", "week", "month", "year", "currentrank"], outputCol="features")

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import RandomForestRegressor,LinearRegression
from pyspark.ml import Pipeline

rf = RandomForestRegressor()
linear = LinearRegression(maxIter=1000)
lr = LogisticRegression(maxIter=1000)
pipeline = Pipeline().setStages([artistInd, titleInd, assembler])

model = pipeline.fit(train)

In [7]:
d = artistInd.fit(train).transform(train)
d = titleInd.fit(d).transform(d)
d = assembler.transform(d)

,artistIndex,titleIndex,weekly_play_count,week,month,year,currentrank
0,1.000000e+00,0.378052,-0.171458,0.130355,1.418347e-01,-7.109485e-08,-0.101029
1,3.780523e-01,1.000000,-0.345547,0.299494,3.248992e-01,0.000000e+00,-0.154544
2,-1.714579e-01,-0.345547,1.000000,-0.015871,-4.861671e-03,0.000000e+00,0.368848
3,1.303549e-01,0.299494,-0.015871,1.000000,9.578186e-01,0.000000e+00,-0.000101
4,1.418347e-01,0.324899,-0.004862,0.957819,1.000000e+00,-1.027276e-07,-0.007906
5,-7.109485e-08,0.000000,0.000000,0.000000,-1.027276e-07,1.000000e+00,0.000000
6,-1.010292e-01,-0.154544,0.368848,-0.000101,-7.905787e-03,0.000000e+00,1.000000


In [107]:
from pyspark.sql.functions import udf, col
results.filter(col("prediction") == 1.0) \
       .sort(col("probability"), ascending=False) \
       .select("artist","prediction", "probability") \
       .take(20)

NameError: name 'results' is not defined

In [ ]:
results = predict(model, test)


In [ ]:
df = get_data(datetime(2017,3,8))
results = predict(model, df)

In [ ]:
df = spark.sql(
"""
SELECT 
    p.artist,
    p.title,
    p.year,
    p.month,
    p.day,
    p.day_of_week,
    p.week,
    COUNT(*) AS daily_play_count
FROM playlist p
GROUP BY
    p.artist,
    p.title,
    p.year,
    p.month,
    p.day,
    p.day_of_week,
    p.week
ORDER BY 
    COUNT(*) DESC
""").show(20)